<a href="https://colab.research.google.com/github/linhctiu/Home-Credit-Default-Risk-VEF/blob/main/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.model_selection import train_test_split
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import pickle
import gc
import lightgbm as lgb
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/application_train.csv')
df2 = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/bureau.csv')
df3 = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/previous_application.csv')
df4 = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/POS_CASH_balance.csv')
df5 = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/installments_payments.csv')
df6 = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/credit_card_balance.csv')


Combine bureau to application_train

In [ ]:
# Combining numerical features
grp_2 = df2.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR']).mean().reset_index()
grp_2.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp_2.columns]
df1_bureau = df1.merge(grp_2, on='SK_ID_CURR', how='left')
# Combining categorical features
df2_categorical = pd.get_dummies(df2.select_dtypes('object'))
df2_categorical['SK_ID_CURR'] = df2['SK_ID_CURR']
grp_2 = df2_categorical.groupby(by = ['SK_ID_CURR']).count().reset_index()
grp_2.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp_2.columns]
df1_bureau = df1_bureau.merge(grp_2, on='SK_ID_CURR', how='left')
# Shape of application and bureau data combined
print('The shape application and bureau data combined:',df1_bureau.shape)

The shape application and bureau data combined: (307511, 157)


In [ ]:
grp = df2.groupby(by = ['SK_ID_CURR'])['SK_ID_BUREAU'].count().reset_index().rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT'})
df1_bureau = df1_bureau.merge(grp, on='SK_ID_CURR', how='left')
df1_bureau['BUREAU_LOAN_COUNT'] = df1_bureau['BUREAU_LOAN_COUNT'].fillna(0)

grp = df2[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
df1_bureau = df1_bureau.merge(grp, on='SK_ID_CURR', how='left')
df1_bureau['BUREAU_LOAN_TYPES'] = df1_bureau['BUREAU_LOAN_TYPES'].fillna(0)

Combine prev.application to application_train

In [ ]:
# Number of previous applications per customer
grp_3 = df3[['SK_ID_CURR','SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV':'PREV_APP_COUNT'})
df1_bureau_prev = df1_bureau.merge(grp_3, on =['SK_ID_CURR'], how = 'left')
df1_bureau_prev['PREV_APP_COUNT'] = df1_bureau_prev['PREV_APP_COUNT'].fillna(0)
# Combining numerical features
grp_3 = df3.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp_3.columns ]
grp_3.columns = prev_columns
df1_bureau_prev = df1_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')

# Combining categorical features
prev_categorical = pd.get_dummies(df3.select_dtypes('object'))
prev_categorical['SK_ID_CURR'] = df3['SK_ID_CURR']
prev_categorical.head()
grp_3 = prev_categorical.groupby('SK_ID_CURR').count().reset_index()
grp_3.columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp_3.columns]
df1_bureau_prev = df1_bureau_prev.merge(grp_3, on=['SK_ID_CURR'], how='left')

Combine POS_CASH_BALANCE to application_train

In [ ]:
# Combining numerical features
grp_4 = df4.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp_4.columns ]
grp_4.columns = prev_columns
df1_bureau_prev = df1_bureau_prev.merge(grp_4, on =['SK_ID_CURR'], how = 'left')

# Combining categorical features
pos_cash_categorical = pd.get_dummies(df4.select_dtypes('object'))
pos_cash_categorical['SK_ID_CURR'] = df4['SK_ID_CURR']
grp_4 = pos_cash_categorical.groupby('SK_ID_CURR').count().reset_index()
grp_4.columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp_4.columns]
df1_bureau_prev = df1_bureau_prev.merge(grp_4, on=['SK_ID_CURR'], how='left')

Combine Installment_payment to application_train


In [ ]:
# Combining numerical features and there are no categorical features in this dataset
grp_5 = df5.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['INSTA_'+column if column != 'SK_ID_CURR' else column for column in grp_5.columns ]
grp_5.columns = prev_columns
df1_bureau_prev = df1_bureau_prev.merge(grp_5, on =['SK_ID_CURR'], how = 'left')


Combine Credit card balance to application_train

In [ ]:
# Combining numerical features
grp_6 = df6.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp_6.columns ]
grp_6.columns = prev_columns
df1_bureau_prev = df1_bureau_prev.merge(grp_6, on =['SK_ID_CURR'], how = 'left')

# Combining categorical features
credit_categorical = pd.get_dummies(df6.select_dtypes('object'))
credit_categorical['SK_ID_CURR'] = df6['SK_ID_CURR']
grp_6 = credit_categorical.groupby('SK_ID_CURR').count().reset_index()
grp_6.columns = ['CREDIT_'+column if column != 'SK_ID_CURR' else column for column in grp_6.columns]
df1_bureau_prev = df1_bureau_prev.merge(grp_6, on=['SK_ID_CURR'], how='left')


In [ ]:
df1_bureau_prev

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,POS_NAME_CONTRACT_STATUS_Approved,POS_NAME_CONTRACT_STATUS_Canceled,POS_NAME_CONTRACT_STATUS_Completed,POS_NAME_CONTRACT_STATUS_Demand,POS_NAME_CONTRACT_STATUS_Returned to the store,POS_NAME_CONTRACT_STATUS_Signed,POS_NAME_CONTRACT_STATUS_XNA,INSTA_NUM_INSTALMENT_VERSION,INSTA_NUM_INSTALMENT_NUMBER,INSTA_DAYS_INSTALMENT,INSTA_DAYS_ENTRY_PAYMENT,INSTA_AMT_INSTALMENT,INSTA_AMT_PAYMENT,CREDIT_MONTHS_BALANCE,CREDIT_AMT_BALANCE,CREDIT_AMT_CREDIT_LIMIT_ACTUAL,CREDIT_AMT_DRAWINGS_ATM_CURRENT,CREDIT_AMT_DRAWINGS_CURRENT,CREDIT_AMT_DRAWINGS_OTHER_CURRENT,CREDIT_AMT_DRAWINGS_POS_CURRENT,CREDIT_AMT_INST_MIN_REGULARITY,CREDIT_AMT_PAYMENT_CURRENT,CREDIT_AMT_PAYMENT_TOTAL_CURRENT,CREDIT_AMT_RECEIVABLE_PRINCIPAL,CREDIT_AMT_RECIVABLE,CREDIT_AMT_TOTAL_RECEIVABLE,CREDIT_CNT_DRAWINGS_ATM_CURRENT,CREDIT_CNT_DRAWINGS_CURRENT,CREDIT_CNT_DRAWINGS_OTHER_CURRENT,CREDIT_CNT_DRAWINGS_POS_CURRENT,CREDIT_CNT_INSTALMENT_MATURE_CUM,CREDIT_SK_DPD,CREDIT_SK_DPD_DEF,CREDIT_NAME_CONTRACT_STATUS_Active,CREDIT_NAME_CONTRACT_STATUS_Approved,CREDIT_NAME_CONTRACT_STATUS_Completed,CREDIT_NAME_CONTRACT_STATUS_Demand,CREDIT_NAME_CONTRACT_STATUS_Refused,CREDIT_NAME_CONTRACT_STATUS_Sent proposal,CREDIT_NAME_CONTRACT_STATUS_Signed
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,19.0,19.0,19.0,19.0,19.0,19.0,19.0,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,28.0,28.0,28.0,28.0,28.0,28.0,28.0,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438,-3.5,0.0,270000.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,66.0,66.0,6

Drop missing columns that have more than 60% missing values

In [ ]:

Missing_values = (df1_bureau_prev.isnull().sum()/len(df1_bureau_prev)*100).sort_values().reset_index()
Missing_columns = Missing_values[Missing_values[0]>60]['index'].values
df1_bureau_prev.drop(columns=Missing_columns,inplace=True)
df1_bureau_prev.drop(columns='WEEKDAY_APPR_PROCESS_START',inplace=True)
df1_bureau_prev.drop_duplicates(inplace=True)

List down categorical after merge

In [ ]:

a2 = list(grp_2.drop(columns=['SK_ID_CURR']).columns)
a3 = list(grp_3.drop(columns=['SK_ID_CURR']).columns)
a4 = list(grp_4.drop(columns=['SK_ID_CURR']).columns)
a6 = list(grp_6.drop(columns=['SK_ID_CURR']).columns)
a = sum([a2,a3,a4,a6], [])
categorical_merge = pd.DataFrame(df1_bureau_prev,columns=a)

Divine data into train,validation and test

In [ ]:
y = df1_bureau_prev.pop('TARGET').values
X_train, X_temp, y_train, y_temp = train_test_split(df1_bureau_prev.drop(['SK_ID_CURR'],axis=1), y, stratify = y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify = y_temp, test_size=0.5, random_state=42)
print('Shape of X_train:',X_train.shape)
print('Shape of X_val:',X_val.shape)
print('Shape of X_test:',X_test.shape)

Shape of X_train: (215257, 303)
Shape of X_val: (46127, 303)
Shape of X_test: (46127, 303)


Processing missing values

In [ ]:
Missing_values = (categorical_merge.isnull().sum()/len(df1_bureau_prev)*100).sort_values(ascending=False).reset_index()
Missing_columns = Missing_values[Missing_values[0]>60]['index'].values
categorical_merge.drop(columns=Missing_columns,inplace=True)

In [ ]:
cat_one_hot_encoding_col = list(categorical_merge.columns)
num_cols = df1_bureau_prev.drop(columns=categorical_merge.columns)._get_numeric_data().drop(columns='SK_ID_CURR')
d = []
for i in num_cols:
  b = df1_bureau_prev[i].unique()
  if len(b)<20:
    d.append(i)
cat_one_hot_encoding_col.extend(d)
num_cols = df1_bureau_prev.drop(columns=cat_one_hot_encoding_col)._get_numeric_data().drop(columns='SK_ID_CURR')
num_cols=num_cols.columns
cat_cols = df1_bureau_prev.drop(columns=num_cols)
cat_cols = cat_cols.drop(columns=cat_one_hot_encoding_col)
cat_cols=cat_cols.drop(columns='SK_ID_CURR')
cat_cols=cat_cols.columns

Extracting numerical data, filling missing values and scaling

In [ ]:
imputer_num = SimpleImputer(strategy='median')
X_train_num = imputer_num.fit_transform(X_train[num_cols])
X_val_num = imputer_num.transform(X_val[num_cols])
X_test_num = imputer_num.transform(X_test[num_cols])

In [ ]:
scaler_num = MinMaxScaler()
X_train_num1 = scaler_num.fit_transform(X_train_num)
X_val_num1 = scaler_num.transform(X_val_num)
X_test_num1 = scaler_num.transform(X_test_num)

In [ ]:
X_train_num_final = pd.DataFrame(X_train_num1, columns=num_cols)
X_val_num_final = pd.DataFrame(X_val_num1, columns=num_cols)
X_test_num_final = pd.DataFrame(X_test_num1, columns=num_cols)

Extracting categorical data, filling missing values and scaling

In [ ]:
# Filling missing values and One hot encoding
imputer_cat = SimpleImputer(strategy="most_frequent",missing_values=np.nan)
X_train_cat = imputer_cat.fit_transform(X_train[cat_cols])
X_val_cat = imputer_cat.transform(X_val[cat_cols])
X_test_cat = imputer_cat.transform(X_test[cat_cols])

In [ ]:
X_train_cat1= pd.DataFrame(X_train_cat, columns=cat_cols)
X_val_cat1= pd.DataFrame(X_val_cat, columns=cat_cols)
X_test_cat1= pd.DataFrame(X_test_cat, columns=cat_cols)
ohe = OneHotEncoder(sparse=False,handle_unknown='ignore')

In [ ]:
X_train_cat2 = ohe.fit_transform(X_train_cat1)
X_val_cat2 = ohe.transform(X_val_cat1)
X_test_cat2 = ohe.transform(X_test_cat1)
cat_cols_ohe = list(ohe.get_feature_names(input_features=cat_cols))

In [ ]:
#Final_categorical_dataframe
X_train_cat_final = pd.DataFrame(X_train_cat2, columns = cat_cols_ohe)
X_val_cat_final = pd.DataFrame(X_val_cat2, columns = cat_cols_ohe)
X_test_cat_final = pd.DataFrame(X_test_cat2, columns = cat_cols_ohe)

Extracting categorical that have been processed by OHE, filling missing values and scaling

In [ ]:
imputer_ohe_cat = SimpleImputer(strategy="most_frequent",missing_values=np.nan)
X_train_OHE_cat = imputer_ohe_cat.fit_transform(X_train[cat_one_hot_encoding_col])
X_val_OHE_cat = imputer_ohe_cat.transform(X_val[cat_one_hot_encoding_col])
X_test_OHE_cat = imputer_ohe_cat.transform(X_test[cat_one_hot_encoding_col])

In [ ]:
X_train_ohe_final = pd.DataFrame(X_train_OHE_cat, columns = cat_one_hot_encoding_col)
X_val_ohe_final = pd.DataFrame(X_val_OHE_cat, columns = cat_one_hot_encoding_col)
X_test_ohe_final = pd.DataFrame(X_test_OHE_cat, columns = cat_one_hot_encoding_col)

Final complete data 

In [ ]:

X_train_final = pd.concat([X_train_num_final,X_train_cat_final,X_train_ohe_final], axis = 1)
X_val_final = pd.concat([X_val_num_final,X_val_cat_final,X_val_ohe_final], axis = 1)
X_test_final = pd.concat([X_test_num_final,X_test_cat_final,X_test_ohe_final], axis = 1)
print(X_train_final.shape)
print(X_val_final.shape)
print(X_test_final.shape)

(215257, 418)
(46127, 418)
(46127, 418)


In [ ]:
X_train_final.to_csv('X_train_final.csv')
!cp X_train_final.csv "drive/My Drive/VEF_ML/PROJECT"
X_val_final.to_csv('X_val_final.csv')
!cp X_val_final.csv "drive/My Drive/VEF_ML/PROJECT"
X_test_final.to_csv('X_test_final.csv')
!cp X_test_final.csv "drive/My Drive/VEF_ML/PROJECT"


In [ ]:
np.savetxt("y_train.csv", y_train, delimiter=",")
!cp y_train.csv "drive/My Drive/VEF_ML/PROJECT"
np.savetxt("y_val.csv", y_val, delimiter=",")
!cp y_val.csv "drive/My Drive/VEF_ML/PROJECT"
np.savetxt("y_test.csv", y_test, delimiter=",")
!cp y_test.csv "drive/My Drive/VEF_ML/PROJECT"

In [ ]:
# cvcv = pd.read_csv('/content/drive/MyDrive/VEF_ML/PROJECT/y_train.csv',header=None)
# cvcv.values.flatten().astype(int)